## 🏡부동산 실거래가 예측 대회👨‍👨‍👧‍👦

### 라이브러리 Import

In [10]:
# Visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', 
    name='NanumBarunGothic')                      
fm.fontManager.ttflist.insert(0, fe)            
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) 
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# Utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [2]:
!pip install eli5==0.13.0

# 한글 폰트 사용을 위한 라이브러리
!apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


### Stage 1. 데이터 확인

In [2]:
# 현재 Directory 경로를 확인 합니다.

import os

print("Current Working Directory:", os.getcwd())

Current Working Directory: /data/ephemeral/home/ML1_DY/code


1. 데이터 불러오기

In [11]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.

train_path = '../data/KOREAN_train4.csv'
test_path  = '../data/KOREAN_test4.csv'
subway_feature_path = '../data/subway_feature.csv'
bus_feature_path = '../data/bus_feature.csv'
sample_submission_path = '../data/sample_submission.csv'
train_geo_path = '../data/train_latlon_complete.csv'

dt_train = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)
dt_subway_feature = pd.read_csv(subway_feature_path)
dt_bus_feature = pd.read_csv(bus_feature_path)
dt_sample_submission = pd.read_csv(sample_submission_path)
dt_train_geo = pd.read_csv(train_geo_path)

In [12]:
# train data set 의 columns를 확인합니다.

dt_train.columns

Index(['시군구', '번지', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명',
       '해제사유발생일', '거래유형', '중개사소재지', 'k-단지분류', 'k-세대타입(분양형태)', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사', 'k-시행사', 'k-연면적', 'k-주거전용면적',
       'k-전용면적별세대현황60이하', 'k-전용면적별세대현황6085이하', 'k-85135이하', 'k-135초과',
       '세대전기계약방법', '건축면적', '주차대수', '기타의무임대1234', '단지신청일', 'target', '구', '동',
       '전체주소명', '동아파트명', '유동인구', '아파트 지하철역 거리', '1차역세권', '2차역세권',
       '아파트 버스정류장 거리', '좌표X', '좌표Y', '아파트 한강대교 거리', '전세가율', '전체계약일자', '금리',
       '실질gdp', '명목gdp', '학군', '재개발개수', '학급수', '여당득표율'],
      dtype='object')

In [13]:
dt_train

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,아파트 한강대교 거리,전세가율,전체계약일자,금리,실질gdp,명목gdp,학군,재개발개수,학급수,여당득표율
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,2823.131513,55.88,20171208,1.50,2.9,"474,957.90",8,0,29.393939,35.36
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,2823.131513,55.88,20171222,1.50,2.9,"474,957.90",8,0,29.393939,35.36
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,2823.131513,55.88,20171228,1.50,2.9,"474,957.90",8,0,29.393939,35.36
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,2823.131513,54.42,20180103,1.50,2.9,"449,357.80",8,0,30.781250,35.36
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,2823.131513,54.42,20180108,1.50,2.9,"449,357.80",8,0,30.781250,35.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118817,서울특별시 은평구 구산동,382,갈현현대,59.94,200707,12,11,1998,서오릉로21길 36,NaN,...,3026.913197,NaN,20070712,4.75,5.5,"275,865.80",2,0,NaN,53.69
1118818,서울특별시 은평구 구산동,382,갈현현대,59.94,200708,25,10,1998,서오릉로21길 36,NaN,...,3026.913197,NaN,20070825,5.00,5.5,"275,865.80",2,0,NaN,53.69
1118819,서울특별시 은평구 구산동,382,갈현현대,84.83,200708,31,20,1998,서오릉로21길 36,NaN,...,3026.913197,NaN,20070831,5.00,5.5,"275,865.80",2,0,NaN,53.69
1118820,서울특별시 은평구 구산동,382,갈현현대,84.83,200709,15,8,1998,서오릉로21길 36,NaN,...,3026.913197,NaN,20070915,5.00,5.5,"275,865.80",2,0,NaN,53.69


각 column 에 대한 입력 값을 추출하기

In [8]:
# 데이터의 각 열에 대한 입력 값 유형, 입력 방법 및 고유 값 수를 찾기 위한 데이터 분석

# 요약 데이터프레임 생성
summary = pd.DataFrame(columns=['변수 이름', '입력 값 예시', '입력 방법', '고유 값의 개수'])

# 각 열에 대한 정보를 수집하기 위해 반복문 실행
for column in dt_train.columns:
    # 데이터 유형과 내용을 기반으로 입력 방법을 '수동' 또는 '자동'으로 추론
    if dt_train[column].dtype == 'object':
        input_method = '수동'
    else:
        input_method = '자동'

    # 요약 데이터프레임에 행 추가
    summary = summary.append({
        '변수 이름': column,
        '입력 값 예시': dt_train[column].dropna().unique()[:5],  # 처음 5개의 고유 비결측치 예시
        '입력 방법': input_method,
        '고유 값의 개수': dt_train[column].nunique()
    }, ignore_index=True)

summary.head()  # 요약 데이터프레임의 처음 몇 행을 표시


,변수 이름,입력 값 예시,입력 방법,고유 값의 개수
0,시군구,"[서울특별시 강남구 개포동, 서울특별시 강남구 논현동, 서울특별시 강남구 대치동, ...",수동,338
1,번지,"[658-1, 652, 12-2, 141, 187]",수동,6457
2,아파트명,"[개포6차우성, 개포우성3차, 개포자이, 개포주공1단지, 개포주공5단지]",수동,6445
3,전용면적,"[79.97, 54.98, 133.46, 161.0, 104.43]",자동,13366
4,계약년월,"[201712, 201801, 201803, 201804, 201806]",자동,117


In [11]:
# Define the file path for the Excel file
csv_file_path = '../data/data_summary.xlsx'

# Writing the dataframe to an Excel file
summary.to_csv(csv_file_path, encoding = 'utf-8', index=False)

csv_file_path

'../data/data_summary.xlsx'

2. 데이터 확인

In [14]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', dt_train.shape, 'Test data shape : ', dt_test.shape)

Train data shape :  (1118822, 53) Test data shape :  (9272, 52)


In [15]:
dt_train.head()

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,아파트 한강대교 거리,전세가율,전체계약일자,금리,실질gdp,명목gdp,학군,재개발개수,학급수,여당득표율
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,8,3,1987,언주로 3,NaN,...,2823.131513,55.88,20171208,1.5,2.9,"474,957.90",8,0,29.393939,35.36
1,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201712,22,4,1987,언주로 3,NaN,...,2823.131513,55.88,20171222,1.5,2.9,"474,957.90",8,0,29.393939,35.36
2,서울특별시 강남구 개포동,658-1,개포6차우성,54.98,201712,28,5,1987,언주로 3,NaN,...,2823.131513,55.88,20171228,1.5,2.9,"474,957.90",8,0,29.393939,35.36
3,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,3,4,1987,언주로 3,NaN,...,2823.131513,54.42,20180103,1.5,2.9,"449,357.80",8,0,30.781250,35.36
4,서울특별시 강남구 개포동,658-1,개포6차우성,79.97,201801,8,2,1987,언주로 3,NaN,...,2823.131513,54.42,20180108,1.5,2.9,"449,357.80",8,0,30.781250,35.36


3. 데이터 타입 확인

In [16]:
dt_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118822 entries, 0 to 1118821
Data columns (total 53 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   시군구                1118822 non-null  object 
 1   번지                 1118822 non-null  object 
 2   아파트명               1116696 non-null  object 
 3   전용면적               1118822 non-null  float64
 4   계약년월               1118822 non-null  int64  
 5   계약일                1118822 non-null  int64  
 6   층                  1118822 non-null  int64  
 7   건축년도               1118822 non-null  int64  
 8   도로명                1118822 non-null  object 
 9   해제사유발생일            5983 non-null     float64
 10  거래유형               32371 non-null    object 
 11  중개사소재지             29241 non-null    object 
 12  k-단지분류             248131 non-null   object 
 13  k-세대타입(분양형태)       249259 non-null   object 
 14  k-복도유형             248932 non-null   object 
 15  k-난방방식             249259 non-nu

In [17]:
numeric_column_names = dt_train.select_dtypes(include='number').columns
numeric_column_names

Index(['전용면적', '계약년월', '계약일', '층', '건축년도', '해제사유발생일', 'k-전체동수', 'k-전체세대수',
       'k-연면적', 'k-주거전용면적', 'k-전용면적별세대현황60이하', 'k-전용면적별세대현황6085이하',
       'k-85135이하', 'k-135초과', '건축면적', '주차대수', 'target', '유동인구', '아파트 지하철역 거리',
       '1차역세권', '2차역세권', '아파트 버스정류장 거리', '좌표X', '좌표Y', '아파트 한강대교 거리', '전세가율',
       '전체계약일자', '금리', '실질gdp', '학군', '재개발개수', '학급수', '여당득표율'],
      dtype='object')

4. 데이터 통계값 확인

- describe() 함수 사용해서 수치형 데이터의 통계값 확인하기

In [19]:
def str_to_int(string):
    if type(string) == str:
        string = string.replace(',', '')
        return int(string)
    else:
        return string

dt_train['target'] = dt_train['target'].apply(str_to_int)

columns = ['전용면적', '층', 'target']
dt_train[columns].describe()

,전용면적,층,target
count,1.118822e+06,1.118822e+06,1.118822e+06
mean,7.717475e+01,8.871968e+00,5.799153e+04
std,2.936423e+01,5.982584e+00,4.642602e+04
min,1.002000e+01,-4.000000e+00,3.500000e+02
25%,5.965000e+01,4.000000e+00,3.050000e+04
50%,8.188000e+01,8.000000e+00,4.480000e+04
75%,8.496000e+01,1.200000e+01,6.980000e+04
max,4.243200e+02,6.900000e+01,1.450000e+06


5. 테스트 데이터 확인

In [20]:
dt_test.head(10)

,시군구,번지,아파트명,전용면적,계약년월,계약일,층,건축년도,도로명,해제사유발생일,...,아파트 한강대교 거리,전세가율,전체계약일자,금리,실질gdp,명목gdp,학군,재개발개수,학급수,여당득표율
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.9700,202307,26,5,1987,언주로 3,NaN,...,2823.131513,41.11,20230726,3.5,1.4,"563,921.50",8,0,33.15625,67.01
1,서울특별시 강남구 개포동,651-1,개포더샵트리에,108.2017,202308,15,10,2021,개포로 311,NaN,...,2340.197569,41.30,20230815,3.5,1.4,"563,921.50",8,0,33.15625,67.01
2,서울특별시 강남구 개포동,652,개포우성3차,161.0000,202307,28,15,1984,개포로 307,NaN,...,2375.597914,41.11,20230728,3.5,1.4,"563,921.50",8,0,33.15625,67.01
3,서울특별시 강남구 개포동,652,개포우성3차,133.4600,202308,10,14,1984,개포로 307,NaN,...,2375.597914,41.30,20230810,3.5,1.4,"563,921.50",8,0,33.15625,67.01
4,서울특별시 강남구 개포동,652,개포우성3차,104.4300,202308,18,6,1984,개포로 307,NaN,...,2375.597914,41.30,20230818,3.5,1.4,"563,921.50",8,0,33.15625,67.01
5,서울특별시 강남구 개포동,187,개포주공5단지,74.2500,202307,28,8,1983,삼성로4길 17,NaN,...,2343.229208,41.11,20230728,3.5,1.4,"563,921.50",8,0,33.15625,67.01
6,서울특별시 강남구 개포동,185,개포주공6단지,83.2100,202307,2,11,1983,개포로 516,NaN,...,2015.573919,41.11,20230702,3.5,1.4,"563,921.50",8,0,33.15625,67.01
7,서울특별시 강남구 개포동,185,개포주공6단지,73.0200,202308,14,12,1983,개포로 516,NaN,...,2015.573919,41.30,20230814,3.5,1.4,"563,921.50",8,0,33.15625,67.01
8,서울특별시 강남구 개포동,185,개포주공6단지,53.0600,202308,24,9,1983,개포로 516,NaN,...,2015.573919,41.30,20230824,3.5,1.4,"563,921.50",8,0,33.15625,67.01
9,서울특별시 강남구 개포동,1280,래미안블레스티지,126.9280,202307,3,26,2019,선릉로 8,NaN,...,2698.919257,41.11,20230703,3.5,1.4,"563,921.50",8,0,33.15625,67.01


6. 제출 데이터 확인

In [21]:
dt_sample_submission.head(10)

,target
0,179048
1,84820
2,248141
3,180991
4,295430
5,229921
6,92951
7,126950
8,240472
9,280156


7. 타겟의 평균값 구하기

In [22]:
def str_to_int(string):
    if type(string) == str:
        string = string.replace(',', '')
        return int(string)
    else:
        return string

dt_train['target'] = dt_train['target'].apply(str_to_int)

mean_apt_price = round(dt_train['target'].mean())
dt_sample_submission['target'] = mean_apt_price
dt_sample_submission.head()

,target
0,57992
1,57992
2,57992
3,57992
4,57992


8. 제출 파일 생성

In [21]:
dt_sample_submission.to_csv('../data/submission_average.csv', index=False)